In [ ]:
import sys, json, time
from threading import Thread, Condition, Event
from psonic import *
sys.path.append("./repo/piplay")
from weather_pilock.SenseHat_Data import Sensor
from pysonic.play_weather import Notes, Rates, rescale
from helpers.helpers import *
import asyncio
import ipywidgets as widgets
from ipywidgets import interact, HBox, VBox
from IPython.display import display

golden_ratio = golden()


In [ ]:
condition = Condition()
stop_event = Event()
rates = Rates()
senseHat = Sensor()

def live_producer():
    sleep(rates.seconds_per_beat)
    pass
def live_loop_producer(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.notifyAll() #Message to threads
        live_producer()
live_thread_producer = Thread(name="producer", target=live_loop_producer, args=(condition, stop_event))

def play_temp():
    pass
def live_loop_temp(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Message to threads
        play_temp()
live_thread_temp = Thread(name="consumer_temp", target=live_loop_temp, args=(condition, stop_event))

def play_pressure():
    pass    
def live_loop_pressure(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Wait for message
        play_pressure()
live_thread_pressure = Thread(name="consumer_pressure", target=live_loop_pressure, args=(condition, stop_event))

def play_humidity():
    pass
def live_loop_humidity(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Wait for message
        play_humidity()
live_thread_humidity = Thread(name="consumer_humidity", target=live_loop_humidity, args=(condition, stop_event))

In [ ]:


start_threads_button = widgets.Button(
    description='Start Threads'
)
def start_threads(b=None):
    global live_thread_producer
    live_thread_producer.start()
    global live_thread_temp
    live_thread_temp.start()
    global live_thread_pressure
    live_thread_pressure.start()
    global live_thread_humidity
    live_thread_humidity.start()     
start_threads_button.on_click(start_threads)

stop_threads_button = widgets.Button(
    description='Stop Threads'
)
def stop_threads(b=None):
    global stop_event
    stop_event.set()
stop_threads_button.on_click(stop_threads)

restart_threads_button = widgets.Button(
    description='Restart Threads'
)
def restart_threads(b=None):
    global stop_event
    stop_event = Event()
    global condition
    condition = Condition()
    global live_loop_producer
    global live_thread_producer
    live_thread_producer = Thread(name="producer", target=live_loop_producer, args=(condition, stop_event))
    live_thread_producer.start()
restart_threads_button.on_click(restart_threads)
    

amp_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description="amp", orientation="horizontal")
pan_slider = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description="pan", orientation="horizontal")
attack_slider = widgets.FloatSlider(value=0.1, min=0.0, max=2.0, step=0.1, description="attack", orientation="horizontal")
decay_slider = widgets.FloatSlider(value=0.0, min=0.0, max=2.0, step=0.1, description="decay", orientation="horizontal")
sustain_slider = widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description="sustain", orientation="horizontal")
sustain_level_slider = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description="sustain level", orientation="vertical")
release_slider = widgets.FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1, description="release", orientation="horizontal")
play_temp_button = widgets.Button(
    description='Update'
)
def play_temp_clicked(b):
    global play_temp
    def play_temp():
        global senseHat
        senseHat.touch()
        temp = Notes([rescale(senseHat.sense.temperature_from_humidity), 
                      rescale(senseHat.sense.temperature_from_pressure)
                     ], 
                     amp = amp_slider.value,
                     pan = pan_slider.value,
                     attack = attack_slider.value, 
                     decay = decay_slider.value,
                     sustain = sustain_slider.value,
                     sustain_level = sustain_level_slider.value,
                     release = release_slider.value
                    )
        play(temp.notes, amp=temp.amp, pan=temp.pan, 
             attack=temp.attack, decay=temp.decay, 
             sustain=temp.sustain, sustain_level=temp.sustain_level, 
             release=temp.release)
        pass
play_temp_button.on_click(play_temp_clicked)

pressure_amp_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description="amp", orientation="horizontal")
pressure_pan_slider = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description="pan", orientation="horizontal")
pressure_attack_slider = widgets.FloatSlider(value=0.1, min=0.0, max=2.0, step=0.1, description="attack", orientation="horizontal")
pressure_decay_slider = widgets.FloatSlider(value=0.0, min=0.0, max=2.0, step=0.1, description="decay", orientation="horizontal")
pressure_sustain_slider = widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description="sustain", orientation="horizontal")
pressure_sustain_level_slider = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description="sustain level", orientation="vertical")
pressure_release_slider = widgets.FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1, description="release", orientation="horizontal")
play_pressure_button = widgets.Button(
    description='Update'
)
def play_pressure_clicked(b):
    global play_pressure
    def play_pressure():
        global senseHat
        senseHat.touch()
        temp = Notes([rescale(senseHat.sense.pressure)], 
                     amp = pressure_amp_slider.value,
                     pan = pressure_pan_slider.value,
                     attack = pressure_attack_slider.value, 
                     decay = pressure_decay_slider.value,
                     sustain = pressure_sustain_slider.value,
                     sustain_level = pressure_sustain_level_slider.value,
                     release = pressure_release_slider.value
                    )
        play(temp.notes, amp=temp.amp, pan=temp.pan, 
             attack=temp.attack, decay=temp.decay, 
             sustain=temp.sustain, sustain_level=temp.sustain_level, 
             release=temp.release)
        pass
play_pressure_button.on_click(play_pressure_clicked)

humidity_amp_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description="amp", orientation="horizontal")
humidity_pan_slider = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description="pan", orientation="horizontal")
humidity_attack_slider = widgets.FloatSlider(value=0.1, min=0.0, max=2.0, step=0.1, description="attack", orientation="horizontal")
humidity_decay_slider = widgets.FloatSlider(value=0.0, min=0.0, max=2.0, step=0.1, description="decay", orientation="horizontal")
humidity_sustain_slider = widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description="sustain", orientation="horizontal")
humidity_sustain_level_slider = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description="sustain level", orientation="vertical")
humidity_release_slider = widgets.FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1, description="release", orientation="horizontal")
play_humidity_button = widgets.Button(
    description='Update'
)
def play_humidity_clicked(b):
    global play_humidity
    def play_humidity():
        global senseHat
        senseHat.touch()
        temp = Notes([rescale(senseHat.sense.humidity)], 
                     amp = humidity_amp_slider.value,
                     pan = humidity_pan_slider.value,
                     attack = humidity_attack_slider.value, 
                     decay = humidity_decay_slider.value,
                     sustain = humidity_sustain_slider.value,
                     sustain_level = humidity_sustain_level_slider.value,
                     release = humidity_release_slider.value
                    )
        play(temp.notes, amp=temp.amp, pan=temp.pan, 
             attack=temp.attack, decay=temp.decay, 
             sustain=temp.sustain, sustain_level=temp.sustain_level, 
             release=temp.release)
        pass
play_humidity_button.on_click(play_humidity_clicked)


In [ ]:
tab_controls = HBox(children=[start_threads_button, stop_threads_button])

tab_temp = HBox(children=[VBox(children=[play_temp_button]), 
                          VBox(children=[amp_slider, pan_slider]),
                          VBox(children=[sustain_level_slider]),
                          VBox(children=[attack_slider, 
                                         decay_slider, 
                                         sustain_slider, 
                                         release_slider
                                        ]
                              )
                         ]
               )
tab_pressure = HBox(children=[VBox(children=[play_pressure_button]), 
                          VBox(children=[pressure_amp_slider, pressure_pan_slider]),
                          VBox(children=[pressure_sustain_level_slider]),
                          VBox(children=[pressure_attack_slider, 
                                         pressure_decay_slider, 
                                         pressure_sustain_slider, 
                                         pressure_release_slider
                                        ]
                              )
                         ]
               )
tab_humidity = HBox(children=[VBox(children=[play_humidity_button]), 
                          VBox(children=[humidity_amp_slider, humidity_pan_slider]),
                          VBox(children=[humidity_sustain_level_slider]),
                          VBox(children=[humidity_attack_slider, 
                                         humidity_decay_slider, 
                                         humidity_sustain_slider, 
                                         humidity_release_slider
                                        ]
                              )
                         ]
               )

tab_sounds = widgets.Tab(children=[tab_temp, tab_pressure, tab_humidity])
tab_sounds.set_title(0, 'Temperature')
tab_sounds.set_title(1, 'Pressure')
tab_sounds.set_title(2, 'Humidity')

VBox(children=[tab_controls, tab_sounds])
